# Time Series Analysis II

In the first lecture, we are mainly concerned with how to model and evaluate time series data.

References

- [Statistical forecasting: notes on regression and time series analysis](https://people.duke.edu/~rnau/411home.htm)
- [Time Series Analysis (TSA) in Python - Linear Models to GARCH](http://www.blackarbs.com/blog/time-series-analysis-in-python-linear-models-to-garch/11/1/2016)

Some Python packages for Time Series modeling

- [`tsai`](https://github.com/timeseriesAI/tsai/tree/main/)
- [`prophet`](https://github.com/facebook/prophet)
- [`statsmodels`](https://github.com/statsmodels/statsmodels)

In [ ]:
import warnings
warnings.simplefilter('ignore', UserWarning)
warnings.simplefilter('ignore', FutureWarning)
warnings.simplefilter('ignore', RuntimeWarning)

## Stationarity

A stationary process is a time series whose mean, variance and auto-covariance do not change over time. Often, transformations can be applied to convert a non-stationary process to a stationary one. Periodicity (seasonality) is another form of non-stationarity that must be accounted for in the modeling.

## Example

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('data/uk-deaths-from-bronchitis-emphys.csv')

In [ ]:
df.head(3)

In [ ]:
df.tail(3)

In [ ]:
df = df.iloc[:-1, :]

In [ ]:
df.columns = ['ds', 'y']

In [ ]:
df['y'] = df['y'].astype('int')

In [ ]:
index = pd.to_datetime(df['ds'], format='%Y-%m').copy()

In [ ]:
df.index = index

In [ ]:
df.index.freq = 'MS'

In [ ]:
df.drop('ds', axis=1, inplace=True)

In [ ]:
df.plot()
pass

### Mean

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(df)
plt.plot(df.rolling(window=12).mean())
pass

### De-trending

We can make the mean stationary by subtracting the trend.

In [ ]:
df1 = df - df.rolling(window=12).mean()
plt.plot(df1)
plt.plot(df1.rolling(window=12).mean())
pass

### Variance

In [ ]:
plt.plot(df.rolling(window=12).var())
pass

### Variance stabilizing transform

It is common to apply a simple variance stabilizing transform, especially if the variance depends on the mean.

In [ ]:
df2 = df.copy()
df2['y'] = np.log(df['y'])

In [ ]:
plt.plot(df2.rolling(window=12).var())
pass

### Periodicity

A simple way to remove periodicity is by differencing. For intuition, consider the simple harmonic oscillator.

$$
\frac{d^2x}{dt^2} = - \omega^2 x
$$

If we look at the second derivative, it is a constant with respect to time (and hence stationary). Differencing twice is a finite approximation to the second derivative, and achieves a similar effect of reducing oscillations.

In [ ]:
from scipy.integrate import odeint

In [ ]:
def f(x, t, ω2):
    y, ydot = x
    return [ydot, -ω2 * y]

In [ ]:
y0 = np.array([0,1])
ts = np.linspace(0, 20, 100)
ω2 = 1

xs = odeint(f, y0, ts, args=(ω2,))

In [ ]:
x = pd.Series(xs[:, 1]) # displacement over time

In [ ]:
x1 = x - x.shift()
x2 = x1 - x1.shift()

In [ ]:
plt.plot(x, label='displacement')
plt.plot(x1, label='velocity')
plt.plot(x2, label='acceleration')
plt.legend()
plt.tight_layout()

### Auto-correlation

The auto-correlation function plots the Pearson correlation between a time series and a lagged version of the same time series.

In [ ]:
[pd.Series.corr(df.y, df.y.shift(i)) for i in range(24)][:3]

For convenience there is also an `autocorr` function

In [ ]:
ac = [df.y.autocorr(i) for i in range(24)]    

In [ ]:
ac[:3]

In [ ]:
plt.stem(ac)
pass

In [ ]:
from pandas.plotting import autocorrelation_plot

In [ ]:
autocorrelation_plot(df.y)
pass

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
plot_acf(df.y, lags=24)
pass

### Partial auto-correlation

The partial auto-correlation at lag $k$ is a conditional correlation, and measures the correlation that remains after taking into account the correlations at lags smaller than $k$. For an analogy, consider the regressions

$y = \beta_0 + \beta_2 x^2$

where $\beta_2$ measures the dependency between $y$ and $x^2$

and

$y = \beta_0 + \beta_1 x + \beta_2 x^2$

where $\beta_2$ measures the dependency between $y$ and $x^2$ after accounting for the dependency between $y$ and $x$.

In [ ]:
plot_pacf(df.y, lags=24)
pass

## Decomposing a model

The simplest models generally decompose the time series into one or more seasonality effects, a trend and the residuals.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
m1 = seasonal_decompose(df)
m1.plot()
pass

In [ ]:
import seaborn as sns
sns.set_context('notebook', font_scale=1.5)

In [ ]:
sns.distplot(m1.resid.dropna().values.squeeze())
pass

## Classical models for time series

### White noise

White noise refers to a time series that is independent and identically distributed (IID) with expectation equal to zero.

In [ ]:
def plot_ts(ts, lags=None):
    fig = plt.figure(figsize = (8, 8))
    ax1 = plt.subplot2grid((2, 2), (0, 0), colspan=2)
    ax2 = plt.subplot2grid((2, 2), (1, 0))
    ax3 = plt.subplot2grid((2, 2), (1, 1))
    
    ax1.plot(ts)
    ax1.plot(ts.rolling(window=lags).mean())
    plot_acf(ts, ax=ax2, lags=lags)
    plot_pacf(ts, ax=ax3, lags=lags)
    
    plt.tight_layout()
    return fig

In [ ]:
np.random.seed(123)
w = pd.Series(np.random.normal(0, 1, 100))

In [ ]:
plot_ts(w, lags=25)
pass

### Random walk

A random walk has the following form

$$
x_t = x_{t-1} + \omega_t
$$

Note that a random walk is not stationary, since there is a time dependence. 

#### Simulate a random walk

In [ ]:
n = 100
x = np.zeros(n)
w = np.random.normal(0, 1, n)

for t in range(n):
    x[t] = x[t-1] + w[t]
    
x = pd.Series(x)

In [ ]:
plot_ts(x, lags=25)

#### Effect of differencing on random walk

Differencing converts a random walk into a white noise process.

In [ ]:
x1 = x - x.shift()
plot_ts(x1.dropna(), lags=25)

### Auto-regressive models of order $p$ AR($p$)

An AR model of order $p$ has the following form

$$
x_t = \sum_{i=1}^p \alpha_i x_{t-i} + \omega_t
$$

where $\omega$ is a white noise term. 

The time series is modeled as a linear combination of past observations.

#### Simulate an AR(1) 

In [ ]:
np.random.seed(123)
n = 300

α = 0.6
x = np.zeros(n)
w = np.random.normal(0, 1, n)

for t in range(n):
    x[t] = α*x[t-1] + w[t]
    
x = pd.Series(x)

In [ ]:
plot_ts(x, lags=25)

Note that a reasonable estimate of $p$ is the largest lag where the partial autocorrelation falls outside the 95% confidence interval. Here it is 1.

#### Fitting an AR model

In [ ]:
from statsmodels.tsa.ar_model import AR

In [ ]:
m2 = AR(x)

In [ ]:
m2.select_order(maxlag=25, ic='aic')

In [ ]:
m2 = m2.fit(maxlag=25, ic='aic')

Compare estimated slope with true slope (=0)

In [ ]:
m2.params[0]

Compare estimated $\alpha$ with treu $\alpha$ (=0.6)

In [ ]:
m2.params[1]

#### Simulate an AR(3) process

In [ ]:
from statsmodels.tsa.api import arma_generate_sample

In [ ]:
np.random.seed(123)

ar = np.array([1, -0.3, 0.4, -0.3])
ma = np.array([1, 0])

x = arma_generate_sample(ar=ar, ma=ma, nsample=100)
x = pd.Series(x)

In [ ]:
plot_ts(x, lags=25)

Note that a reasonable estimate of $p$ is the largest lag where the partial autocorrelation consistently falls outside the 95% confidence interval. Here it is 3

#### Fitting an AR model

In [ ]:
from statsmodels.tools.sm_exceptions import HessianInversionWarning, ConvergenceWarning
import warnings
warnings.simplefilter('ignore', HessianInversionWarning)
warnings.simplefilter('ignore', ConvergenceWarning)

In [ ]:
m3 = AR(x)
m3.select_order(maxlag=25, ic='aic')

In [ ]:
m3 = m3.fit(maxlag=25, ic='aic')

Compare with true coefficients (0.3, -0.4, 0.3)

In [ ]:
m3.params[1:4]

## Moving Average models MA(q)

A moving average model of order $q$ is

$$
x_t = \sum_{i=1}^{q} \beta_i w_{t-i} + w_t
$$

The time series is modeled as a linear combination of past white noise terms.

#### Simulate MA(3) 

In [ ]:
ar = np.array([1, 0])
ma = np.array([1, 0.3, 0.4, 0.7])

x = arma_generate_sample(ar=ar, ma=ma, nsample=100)
x = pd.Series(x)

In [ ]:
plot_ts(x, lags=25)
pass

Note that a reasonable estimate of $q$ is the largest lag where the autocorrelation falls outside the 95% confidence interval. Here it is probably between 3 and 5.

#### Fit an MA model

In [ ]:
from statsmodels.tsa.arima_model import ARMA

In [ ]:
p = 0
q = 3
m4 = ARMA(x, order=(p, q))
m4 = m4.fit(maxlag=25, method='mle')

Compare with true coefficients (0.3, 0.4, 0.7)

In [ ]:
m4.params[1:4]

In [ ]:
m4.summary2()

## ARMA(p, q)

As you might have suspected, we can combine the AR and MA models to get an ARMA model. The ARMA model takes the form

$$
x_t = \sum_{i=1}^{p} \alpha_i x_{t-i} + \sum_{i=1}^{q} \beta_i w_{t-i} + w_t
$$

In [ ]:
np.random.seed(123)

ar = np.array([1, -0.3, 0.4, -0.3])
ma = np.array([1, 0.3, 0.4, 0.7])

x = arma_generate_sample(ar=ar, ma=ma, nsample=100)
x = pd.Series(x)

In [ ]:
plot_ts(x, lags=25)
pass

In [ ]:
p = 3
q = 3
m5 = ARMA(x, order=(p, q))
m5 = m5.fit(maxlag=25, method='mle')

In [ ]:
m5.summary2()

#### Estimating order

We can loop through a range of orders (inspect the ACF and PACF plots) to choose the order for the AR model.

In [ ]:
best_aic = np.infty

for p in np.arange(5):
    for q in np.arange(5):
        try:
            # We assume that the data has been detrended
            m_ = ARMA(x, order=(p, q)).fit(method='mle', trend='nc') 
            aic_ = m_.aic
            if aic_ < best_aic:
                best_aic = aic_
                best_order = (p, q)
                best_m = m_
        except:
            pass

In [ ]:
best_order

### ARIMA(p, d, q)

The ARIMA model adds differencing to convert a non-stationary model to stationarity. The parameter $d$ is the number of differencings to perform.

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
best_aic = np.infty

for p in np.arange(5):
    for d in np.arange(3):
        for q in np.arange(5):
            try:
                # We assume that the data has been detrended
                m_ = ARIMA(x, order=(p, d, q)).fit(method='mle', trend='nc') 
                aic_ = m_.aic
                if aic_ < best_aic:
                    best_aic = aic_
                    best_order = (p, d, q)
                    best_m = m_
            except:
                pass

In [ ]:
best_order

### ARMA on UK disease data

In [ ]:
df.head()

In [ ]:
plot_ts(df, lags=25)

In [ ]:
best_aic = np.infty

for p in np.arange(5, 10):
    for q in np.arange(5, 10):
        try:
            m_ = ARMA(df, order=(p, q)).fit(method='mle')
            aic_ = m_.aic
            if aic_ < best_aic:
                best_aic = aic_
                best_order = (p, q)
                best_m = m_
        except:
            pass

In [ ]:
best_order

#### Fit ARMA model

In [ ]:
m6 = ARMA(df, order=best_order)
m6 = m6.fit(maxlag=25, method='mle')

In [ ]:
m6.summary2()

### Making forecasts

In [ ]:
y_pred = m6.predict(df.index[0], df.index[-1] + pd.Timedelta(1, unit='D') )

In [ ]:
fig = plot_ts(y_pred, lags=25)
fig.axes[0].axvline(df.index[-1], c='red')
pass

## Bayesian modeling with `prophet`

```bash
! python3 -m pip install --quiet fbprophet
```

In [ ]:
from fbprophet import Prophet

Data needs to have just two columns `ds` and `y`.

In [ ]:
data = df.reset_index()

In [ ]:
m7 = Prophet(weekly_seasonality=False, daily_seasonality=False)

In [ ]:
m7 = m7.fit(data)

#### Making forecasts

In [ ]:
future = m7.make_future_dataframe(periods=24, freq='M')

In [ ]:
forecast = m7.predict(future)

In [ ]:
m7.plot(forecast)
pass

## Model evaluation

### Similarity measures

There are several measures commonly used to evaluate the quality of forecasts. The are the same measures we use to evaluate the fit to any function such as$R^2$, MSE and MAE, so will not be described further here. 

### Cross-validation

From https://cdn-images-1.medium.com/max/800/1*6ujHlGolRTGvspeUDRe1EA.png

![img](https://cdn-images-1.medium.com/max/800/1*6ujHlGolRTGvspeUDRe1EA.png)

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

In [ ]:
tsp = TimeSeriesSplit(n_splits=3)

In [ ]:
for train, test in tsp.split(df):
    print(train.shape, test.shape)

In [ ]:
df.index[train[-1]]

In [ ]:
df.index[train[-1]+len(test)]

#### A routine like the following can be used for model comparison

In [ ]:
res = []
for train, test in tsp.split(df):
    m = Prophet(yearly_seasonality=False, 
                weekly_seasonality=False, 
                daily_seasonality=False)
    m.fit(data.iloc[train])
    future = data[['ds']].iloc[test]
    y_pred = m.predict(future).yhat
    y_true = data.y[test]
    res.append(mean_squared_error(y_true, y_pred))

In [ ]:
np.mean(res)

#### If you are using `prophet` it includees its own [diagnostic functions](https://facebook.github.io/prophet/docs/diagnostics.html)

Prophet is oriented for daily data. At present, it does not appear to support cross-validation for monthly data.